In [ ]:
# Импорт библиотек
import os
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats 
from sklearn.preprocessing import QuantileTransformer, PowerTransformer, RobustScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import pickle
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

# многострочный вывод без использования print
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# pip freeze > requirements.txt

In [ ]:
# Определим пути откуда будет загружать файлы
current_path = os.getcwd()
file_path1 = current_path+'\content\Composites\X_bp.xlsx'
file_path2 = current_path+'\content\Composites\X_nup.xlsx'

In [ ]:
# Загружаем первый файл
df1 = pd.read_excel(file_path1, index_col = 0) 
df1.shape 
df1.head(5)

In [ ]:
# Загружаем второй файл
df2 = pd.read_excel(file_path2, index_col = 0)
df2.shape
df2.tail(5)

In [ ]:
# Объединение таблиц 
df_united = df1.join(df2, how='inner') 
df_united.shape
df_united.sample(5).T
# '''
# Таблицы имели разную размерность, но при объединении с типом inner: 
#     было выполнено внутреннее соединение - объединились только строки, имеющие одинаковый индекс  
# '''

In [ ]:
# Поменяем название одного столбца, сделаем с заглавной)) 
df_united.rename(columns = {'модуль упругости, ГПа':'Модуль упругости, ГПа'}, inplace = True)

# Первичный анализ

In [ ]:
# На всякий случай удалим дубликаты (если такие имеются)
df_united.drop_duplicates(inplace = True)
df_united.shape

In [ ]:
# Посмотрим суммарную информацию 
df_united.info()

## Описание столбцов

    Соотношение матрица-наполнитель       - Цель № 3      
    Плотность, кг/м3                      - характеристика матрицы   
    Модуль упругости, ГПа                 - характеристика матрицы 
    Количество отвердителя, м.%           - характеристика матрицы 
    Содержание эпоксидных групп,%_2       - характеристика матрицы 
    Температура вспышки, С_2              - характеристика матрицы  
    Поверхностная плотность, г/м2         - характеристика матрицы 
    Модуль упругости при растяжении, ГПа  - Цель № 1 
    Прочность при растяжении, МПа         - Цель № 2 
    Потребление смолы, г/м2               - характеристика наполнителя
    Угол нашивки, град                    - характеристика наполнителя     
    Шаг нашивки                           - характеристика наполнителя   
    Плотность нашивки                     - характеристика наполнителя    

In [ ]:
# Дополнительно проверим на пустые значения 
df_united.isnull().any() 
# df.isnull().count() 
# df.isna().count()

In [ ]:
# Проверим уникальные значения по столбцам
df_united.nunique()

In [ ]:
# Выведем описательные статистические данные
df_united.describe(include = 'all').T

In [ ]:
# Посмотрим моду
df_united.mode().iloc[0]

In [ ]:
# Посмотрим медиану
df_united.median()

In [ ]:
# Добавим моду и медиану к статданным  
def get_df_stat(dataset):
    df_stat_1 = dataset.describe()
    num_col = [i for i in dataset.columns if dataset[i].dtype in ['int64','float64']]
    df_stat_2 = pd.DataFrame(dataset[num_col].median()).T.rename(index = {0:'median'})
    df_stat_3 = pd.DataFrame(dataset[num_col].mode().iloc[0]).T.rename(index = {0:'mode'})
    df_stat = pd.concat([df_stat_1, df_stat_2, df_stat_3])
    return df_stat

In [ ]:
get_df_stat(df_united).T

In [ ]:
# Посмотрим корреляционную матрицу 
df_united.corr(method = 'pearson') 

In [ ]:
# Визуализируем корреляционную матрицу 
sns.heatmap(df_united.corr(method = 'pearson'), 
            annot=True, 
            fmt ='.1f', 
            cmap= 'coolwarm',
            linewidths=0.1, 
            linecolor='black')

In [ ]:
# Выведем графики распределения каждой из переменной, попарные графики рассеяния точек
sns.pairplot(data = df_united, kind = 'scatter', diag_kind = 'kde')

In [ ]:
# Сделаем функцию для вывоода графиков: histplot, boxplot, Q-Q
def draw_diagram (dataset, column, stat_dataset = None):
    fig = plt.figure(figsize = (20,5))
    
    Q1 = dataset[column].quantile(0.25)
    Q3 = dataset[column].quantile(0.75)
    IQR = Q3 - Q1
    Qmin = (Q1 - 1.5 * IQR)
    Qmax = (Q3 + 1.5 * IQR)
    skew = stats.skew(dataset[column], axis=0, bias=True)
    
    # гистограмма распределения
    plt.subplot(1, 3, 1)
    sns.histplot(x = dataset[column], kde = True, color = 'orange')
    
    if stat_dataset is not None:
        plt.axvline(stat_dataset.loc['mean',column], 
                    c ='red', label = 'mean', 
                    lw = 2, ls = '--', ymax = 0.85)
        plt.axvline(stat_dataset.loc['median',column], 
                    c = 'blue', label = 'median', 
                    lw = 2, ls = '-', ymax = 0.85)
        plt.axvline(stat_dataset.loc['mode',column], 
                    c = 'green', label = 'mode', 
                    lw = 2, ls = '--', ymax = 0.85)
        plt.legend(loc='best')
    plt.axvline(Qmin, 
                c ='black', label = 'Q1 - 1.5 * IQR', 
                lw = 2, ls = '-', ymin = 0, ymax = 0.05)   
    plt.axvline(Q1, 
                c ='black', label = 'Q1', 
                lw = 3, ls = '-', ymin = 0, ymax = 0.1)
    plt.axvline(Q3, 
                c ='black', label = 'Q3', 
                lw = 3, ls = '-', ymin = 0, ymax = 0.1)
    plt.axvline(Qmax, 
                c ='black', label = 'Q3 + 1.5 * IQR', 
                lw = 2, ls = '-', ymin = 0, ymax = 0.05)
    plt.legend(loc='best')
    plt.title(f'skew = {round(skew,3)}')
    plt.xlabel(None)
    plt.ylabel(None)    
    
    # ящик с усами
    plt.subplot(1, 3, 2)
    sns.boxplot(x = dataset[column], orient = "h", color = 'chartreuse') 
    plt.title(column)
    plt.xlabel(None)

    # график Q-Q
    plt.subplot(1, 3, 3)
    stats.probplot(x = dataset[column], dist="norm", plot=plt) 
    plt.title(None)
    plt.xlabel(None)
    plt.ylabel(None)
    
    plt.show()

In [ ]:
for i in df_united.columns:
    if df_united[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_united, i, get_df_stat(df_united))

По графикам видим:
 - наличие выбросов
 - распеределение близкое к нормальному по всем параметрам, параметр "Поверхностная плотность, г/м2" имеет положительную ассиметрию 

In [ ]:
# Посмотрим подробные отчеты с помощью ProfileReport
ProfileReport(df_united)

In [ ]:
# '''
# Параметр 'Угол нашивки, град' преобразуем в категориальный признак или нет?
# По данным из имеющегося датасета данный параметр принимает только два значения.
# Но: скорее всего градус угла может быть отличным от указанных значений 
# '''

# Поменяем type
df_united_cat = df_united.copy()
df_united_cat['Угол нашивки, град'] = df_united_cat['Угол нашивки, град'].astype('category')

# Кодирование категорий с помощью skclearn OneHotEncoder
df_united_OHE = df_united.copy()
OHE = OneHotEncoder()
OHE = OHE.fit(df_united_OHE[['Угол нашивки, град']])
df_united_OHE = df_united_OHE.join(pd.DataFrame(OHE.transform(df_united_OHE[['Угол нашивки, град']]).toarray()))
df_united_OHE.drop('Угол нашивки, град', axis = 1, inplace = True)

# Кодирование категорий с помощью Pandas_get_dummies
df_united_GD = df_united.copy()
df_united_GD = pd.get_dummies(df_united_GD, columns=['Угол нашивки, град'])

# Борьба с выбросами


#### Каким способом будем бороться с выбросами:
 1. Удалим используя правила трех сигм - '3sig'
 2. Удалим используя межквартильный размах - 'IQR'
 3. Удалим используя 5% и 95% квантилей - '5Q95'
 4. Оставим
 

In [ ]:
# Функция удаления выбросов
def drop_blowout (dataset, drop_method = None, exp_col = None):
    
    if drop_method is not None:      
        print (f'Метод удаления - {drop_method}')
        if exp_col is not None:  
            num_col = [i for i in dataset.columns if dataset[i].dtype in ['int64','float64'] and i not in exp_col]
        else:
            num_col = [i for i in dataset.columns if dataset[i].dtype in ['int64','float64']]        
        index_drop = []
        
        for col_name in num_col:             
            
            if drop_method == '3sig':
                border_low = dataset[col_name].mean() - (3 * dataset[col_name].std())
                border_up = dataset[col_name].mean() + (3 * dataset[col_name].std())
                
            elif drop_method == 'IQR':
                IQR = dataset[col_name].quantile(0.75) - dataset[col_name].quantile(0.25)
                border_low = dataset[col_name].quantile(0.25) - (1.5 * IQR)
                border_up = dataset[col_name].quantile(0.75) + (1.5 * IQR)   
                
            elif drop_method == '5Q95':
                border_low = dataset[col_name].quantile(0.05)
                border_up = dataset[col_name].quantile(0.95)

            index_line = dataset[(dataset[col_name] < border_low) | (dataset[col_name] > border_up)].index  
            index_drop.extend(index_line)
            print(f'Для параметра: "{col_name}": удалено {len(index_line)} строк с индексами {list(index_line)}') 
            
        print(f'Будет удалено {len(list(set(index_drop)))} строк с индексами {set(index_drop)}')
        dataset_new = dataset.drop(index = list(set(index_drop)), axis = 0) 
        print(f'Новый размер = {dataset_new.shape}')
        return dataset_new
    else: 
        return dataset 

In [ ]:
df_3sig = drop_blowout(df_united_cat, '3sig')
print()
df_3sig = drop_blowout(df_3sig, '3sig')
# print()
# df_3sig = drop_blowout(df_3sig, '3sig')

In [ ]:
for i in df_3sig.columns:
    if df_3sig[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_3sig, i, get_df_stat(df_3sig))

In [ ]:
df_IQR = drop_blowout(df_united_cat, 'IQR') 
print()
df_IQR = drop_blowout(df_IQR, 'IQR')
print()
df_IQR = drop_blowout(df_IQR, 'IQR')
# print()
# df_IQR = drop_blowout(df_IQR, 'IQR')

In [ ]:
for i in df_IQR.columns:
    if df_IQR[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_IQR, i, get_df_stat(df_IQR))

In [ ]:
df_5Q95 = drop_blowout(df_united_cat, '5Q95')

In [ ]:
for i in df_5Q95.columns:
    if df_5Q95[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_5Q95, i, get_df_stat(df_5Q95))


#### Что получили:  
 1. Используя правила трех сигм - '3sig' - удалили 23 строки, при повторных итерациях еще 4 
 2. Используя межквартильный размах - 'IQR'- удалии 87 строк, при повторных итерациях еще 14
 3. Используя 5% и 95% квантилей - '5Q95' - удалили 727 строк (пока пропустим данный метод по причине большой потери данных + прослеживается некая закономерность в подмене данных, нужно вернуться к данному моменту позже)
 

# Преобразование данных


#### Каким способом будет преобразовывать данные: 
 1. StandardScaler - для датасетов с удалением выбросов, кроме распределений с явной ассиметрией 
    1.  PowerTransformer - для датасетов с удалением выбросов и распределений с явной ассиметрией  
 2. MinMaxScaler - для датасетов с удалением выбросов
 3. RobustScaler, QuantileTransformer - для датасетов без удаления выбросов 
 
Целевые переменные трогать не будем


In [ ]:
# Попробуем сделать логарифмическое преобразование "Поверхностная плотность, г/м2"
df_norm_lg = df_united.copy()
df_norm_lg['Поверхностная плотность, г/м2'] = np.log(df_norm_lg['Поверхностная плотность, г/м2'])
draw_diagram(df_norm_lg, 'Поверхностная плотность, г/м2')

In [ ]:
# Попробуем сделать преобразование "Поверхностная плотность, г/м2" через kвадратный корень 
df_norm_sqrt = df_united.copy()
df_norm_sqrt['Поверхностная плотность, г/м2'] = df_norm_sqrt['Поверхностная плотность, г/м2']**(1/2)
draw_diagram(df_norm_sqrt, 'Поверхностная плотность, г/м2')

In [ ]:
# Попробуем сделать преобразование "Поверхностная плотность, г/м2" через возведение в степень 
df_norm_power = df_united.copy()
df_norm_power['Поверхностная плотность, г/м2'] = df_norm_power['Поверхностная плотность, г/м2']**2
draw_diagram(df_norm_power, 'Поверхностная плотность, г/м2',)

In [ ]:
# Попробуем сделать преобразование с помощью skclearn PowerTransformer методом Йео-Джонсона
PowerTranYJ = PowerTransformer(method = 'yeo-johnson', standardize = False)
df_norm_PTYJ = df_united.copy()
df_norm_PTYJ['Поверхностная плотность, г/м2'] = PowerTranYJ.fit_transform(np.ravel(df_norm_PTYJ['Поверхностная плотность, г/м2']).reshape(-1, 1))
draw_diagram(df_norm_PTYJ, 'Поверхностная плотность, г/м2', get_df_stat(df_norm_PTYJ))

In [ ]:
# Попробуем сделать преобразование с помощью scipy методом Йео-Джонсона
df_norm_sc_YJ = df_united.copy()
df_norm_sc_YJ['Поверхностная плотность, г/м2'], paramyj = stats.yeojohnson(df_norm_sc_YJ['Поверхностная плотность, г/м2'])
draw_diagram(df_norm_sc_YJ, 'Поверхностная плотность, г/м2', get_df_stat(df_norm_sc_YJ))

In [ ]:
# Попробуем сделать преобразование с помощью skclearn PowerTransformer методом Бокса-Кокса
PowerTranBC = PowerTransformer(method = 'box-cox', standardize = False)
df_norm_PTBC = df_united.copy()
df_norm_PTBC['Поверхностная плотность, г/м2'] = PowerTranBC.fit_transform(np.ravel(df_norm_PTBC['Поверхностная плотность, г/м2']).reshape(-1, 1))
draw_diagram(df_norm_PTBC, 'Поверхностная плотность, г/м2', get_df_stat(df_norm_PTBC))

In [ ]:
# Попробуем сделать преобразование с помощью scipy методом Бокса-Кокса
df_norm_sc_BC = df_united.copy()
df_norm_sc_BC['Поверхностная плотность, г/м2'], parambc = stats.boxcox(df_norm_sc_BC['Поверхностная плотность, г/м2'])
draw_diagram(df_norm_sc_BC, 'Поверхностная плотность, г/м2', get_df_stat(df_norm_sc_BC))

In [ ]:
# Попробуем сделать преобразование с помощью skclearn QuantileTransformer
QuanTran = QuantileTransformer(output_distribution='normal')
df_norm_QT = df_united.copy()
df_norm_QT['Поверхностная плотность, г/м2'] = QuanTran.fit_transform(np.ravel(df_norm_QT['Поверхностная плотность, г/м2']).reshape(-1, 1))
draw_diagram(df_norm_QT, 'Поверхностная плотность, г/м2', get_df_stat(df_norm_QT))

In [ ]:
# Оперделим целевые переменные, чтоб исключить из нормализации 
target_col = ['Соотношение матрица-наполнитель','Модуль упругости при растяжении, ГПа','Прочность при растяжении, МПа'] 

In [ ]:
# Функция преобразования
def dataset_normal(dataset, normal_method, exp_col = None, pw_col = None):
    
    dataset_normal = dataset.copy()
    
    if exp_col is not None:
        num_col = [i for i in df_united_cat.columns if df_united_cat[i].dtype in ['int64','float64'] and i not in exp_col]
    else:
        num_col = [i for i in df_united_cat.columns if df_united_cat[i].dtype in ['int64','float64']]
    
    if normal_method == 'MinMaxScaler':
        MMS = MinMaxScaler()
        dataset_normal[num_col] = MMS.fit_transform(dataset_normal[num_col])

    elif normal_method == 'RobustScaler':
        RS = RobustScaler()
        dataset_normal[num_col] = RS.fit_transform(dataset_normal[num_col])

    elif normal_method == 'QuantileTransformer':
        QT = QuantileTransformer(output_distribution='normal', n_quantiles = len(dataset))
        dataset_normal[num_col] = QT.fit_transform(dataset_normal[num_col])
    
    elif normal_method == 'StandardScaler':
            
        if pw_col is not None:
            num_col.remove(pw_col)
            PT = PowerTransformer(method = 'yeo-johnson')
            dataset_normal[pw_col] = PT.fit_transform(np.ravel(dataset_normal[pw_col]).reshape(-1, 1))            
        SC = StandardScaler()
        dataset_normal[num_col] = SC.fit_transform(dataset_normal[num_col])   

    return dataset_normal               


#### Текущие датасеты
 0. df_united     - первичный датасет 
 1. df_united_cat - датасет, параметр "Угол нашивки" переведен в категориальный
 2. df_3sig       - датасет с удаление выбросов с помощью правил 3 сигм
 3. df_IQR        - датасет с удаление выбросов с помощью межквартильного размаха
 4. df_5Q95       - датасет с удаление выбросов с помощью 5% и 95% квантилей
 

In [ ]:
list_dataset = [] # будем хранит список наших датасетов

In [ ]:
df_3sig_StandardScaler = dataset_normal(df_3sig, 'StandardScaler', target_col, 'Поверхностная плотность, г/м2')
for i in df_3sig_StandardScaler.columns:
    if df_3sig_StandardScaler[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_3sig_StandardScaler, i, get_df_stat(df_3sig_StandardScaler))

In [ ]:
df_3sig_StandardScaler.index.name = '3sig_SC'
list_dataset.append(df_3sig_StandardScaler)        

In [ ]:
df_3sig_MinMaxScaler = dataset_normal(df_3sig, 'MinMaxScaler', target_col)
for i in df_3sig_MinMaxScaler.columns:
    if df_3sig_MinMaxScaler[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_3sig_MinMaxScaler, i, get_df_stat(df_3sig_MinMaxScaler))

In [ ]:
df_3sig_MinMaxScaler.index.name = '3sig_MMS'
list_dataset.append(df_3sig_MinMaxScaler)        

In [ ]:
df_IQR_StandardScaler = dataset_normal(df_IQR, 'StandardScaler', target_col, 'Поверхностная плотность, г/м2')
for i in df_IQR_StandardScaler.columns:
    if df_IQR_StandardScaler[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_IQR_StandardScaler, i, get_df_stat(df_IQR_StandardScaler))

In [ ]:
df_IQR_StandardScaler.index.name = 'IQR_SC'
list_dataset.append(df_IQR_StandardScaler)    

In [ ]:
df_IQR_MinMaxScaler = dataset_normal(df_IQR, 'MinMaxScaler', target_col)
for i in df_IQR_MinMaxScaler.columns:
    if df_IQR_MinMaxScaler[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_IQR_MinMaxScaler, i, get_df_stat(df_IQR_MinMaxScaler))

In [ ]:
df_IQR_MinMaxScaler.index.name = 'IQR_MMS'
list_dataset.append(df_IQR_MinMaxScaler)  

In [ ]:
df_5Q95_StandardScaler = dataset_normal(df_5Q95, 'StandardScaler', target_col, 'Поверхностная плотность, г/м2')
for i in df_5Q95_StandardScaler.columns:
    if df_5Q95_StandardScaler[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_5Q95_StandardScaler, i, get_df_stat(df_5Q95_StandardScaler))

In [ ]:
df_5Q95_StandardScaler.index.name = '5Q95_SC'
list_dataset.append(df_5Q95_StandardScaler) 

In [ ]:
df_5Q95_MinMaxScaler = dataset_normal(df_5Q95, 'MinMaxScaler', target_col)
for i in df_5Q95_MinMaxScaler.columns:
    if df_5Q95_MinMaxScaler[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_5Q95_MinMaxScaler, i, get_df_stat(df_5Q95_MinMaxScaler))

In [ ]:
df_5Q95_MinMaxScaler.index.name = '5Q95_MMS'
list_dataset.append(df_5Q95_MinMaxScaler)  

In [ ]:
df_all_RobustScaler = dataset_normal(df_united_cat, 'RobustScaler', target_col)
for i in df_all_RobustScaler.columns:
    if df_all_RobustScaler[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_all_RobustScaler, i, get_df_stat(df_all_RobustScaler))

In [ ]:
df_all_RobustScaler.index.name = 'all_RS'
list_dataset.append(df_all_RobustScaler)  

In [ ]:
df_all_QuantileTransformer = dataset_normal(df_united_cat, 'QuantileTransformer', target_col)
for i in df_all_QuantileTransformer.columns:
    if df_all_QuantileTransformer[i].dtype in ['int64', 'float64']:       
        draw_diagram(df_all_QuantileTransformer, i, get_df_stat(df_all_QuantileTransformer))

In [ ]:
df_all_QuantileTransformer.index.name = 'all_QT'
list_dataset.append(df_all_QuantileTransformer)  

In [ ]:
for i in range(0,len(list_dataset)):
    print(list_dataset[i].index.name) 

# Выбор датасета для работы

#### Попробуем сделать подбор модели для решения задачи регрессии: 
    
    1. LinearRegression - обычная линейная регрессия методом наименьших квадратов 
    2. DecisionTreeRegressor - дерево решений
    3. RandomForestRegressor - случайный лес
    4. LinearSVR - метод опорных векторов
    5. KNeighborsRegressor - метод ближайших соседей 

Оценку будем производить с помощью среднеквадратичной ошибки и коэффициента детерминации

In [ ]:
target_col

In [ ]:
# Сделаем словарь для удобства вызова имени целевой переменной
dict_target = {
              'Соотношение М-Н' : 'Соотношение матрица-наполнитель', 
              'Модуль упругости' : 'Модуль упругости при растяжении, ГПа',
              'Прочность' : 'Прочность при растяжении, МПа'
              }

# Сделаем словарь для удобства вызова модели, параметры оставим по умолчанию
dict_model = {
              'LinReg' : LinearRegression(), 
              'DecTreeReg' : DecisionTreeRegressor(random_state = 13),
              'LinSVR': LinearSVR(random_state = 13),
              'KNNReg': KNeighborsRegressor(),              
              'RandForReg' : RandomForestRegressor(random_state = 13)
             }

In [ ]:
result_dataset = []
for i in range(0,len(list_dataset)):
    
    X_all = list_dataset[i]
    X_all = pd.get_dummies(X_all, columns=['Угол нашивки, град'])
    result_target = []
    
    for target_name, target in dict_target.items(): 
        
        result_model = []
        y = X_all[target]
        X = X_all.drop(target_col, axis = 1)
        X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state = 13)
        
        for model_name, model in dict_model.items():           
            model = model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            result_model.append([list_dataset[i].index.name, target_name, model_name, mae, mse, r2])  
        else:
            result_model = pd.DataFrame(result_model, columns = ['Dataset', 'Target', 'Model', 'MAE', 'MSE', 'R2']) 
            result_target.append(result_model)
    else:
        result_dataset.append(result_target)

In [ ]:
# Поличили оценки прогноза каждой модели по каждому датасету
for i in range(0,len(list_dataset)):
    for g in range(0,len(target_col)):
        result_dataset[i][g]

#### Выводы:

1 Для датасета (удаление выбросов - правило 3 сигм, преобразование - StandardScaler) - наилучшие показатели для всех трех целевых переменных у обычной линейной регрессии  

2 Для датасета (удаление выбросов - правило 3 сигм, преобразование - MinMaxScaler) - наилучшие показатели для всех трех целевых переменных у обычной линейной регрессии
    
3 Для датасета (удаление выбросов - межквартильный размах, преобразование - StandardScaler) - наилучшие показатели для всех трех целевых переменных у обычной линейной регрессии 
  
4 Для датасета (удаление выбросов - межквартильный размах, преобразование - MinMaxScaler) - наилучшие показатели для всех трех целевых переменных у обычной линейной регрессии 
    
5 Для датасета (удаление выбросов -  5% и 95% квантилей, преобразование - StandardScaler) - наилучшие показатели для:
- соотношение матрица-наполнитель -  метод опорных векторов и обычная линейная регрессия 
- модуль упругости и прочность - обычная линейная регрессия
        
6 Для датасета (удаление выбросов - 5% и 95% квантилей, преобразование - MinMaxScaler) - наилучшие показатели для всех трех целевых переменных у обычной линейной регрессии        
    
7 Для датасета (без удаления выбросов, преобразование - RobustScaler) - наилучшие показатели для:
   - соотношение матрица-наполнитель -  метод опорных векторов и обычная линейная регрессия 
   - модуль упругости и прочность - обычная линейная регрессия    
        
8 Для датасета (без удаления выбросов, преобразование - QuantileTransformer) - наилучшие показатели для:
   - соотношение матрица-наполнитель -  метод опорных векторов и обычная линейная регрессия     
   - модуль упругости и прочность - обычная линейная регрессия       
      
Так как метод обычной регрессии показал наилучшие результаты (если можно так сказать) - можно сравнить результаты между ними, чтоб определится с каким датасетом будем работать дальше   

In [ ]:
result_dataset_itog = []
for i in range(0,len(list_dataset)):
    
    X_all = list_dataset[i]
    X_all = pd.get_dummies(X_all, columns=['Угол нашивки, град'])
    result_target = []
    
    for target_name, target in dict_target.items(): 
        
        y = X_all[target]
        X = X_all.drop(target_col, axis = 1)
        X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state = 13)
      
        model = dict_model['LinReg']           
        model_name = 'LinReg'
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        result_dataset_itog.append([list_dataset[i].index.name, target_name, model_name, mae, mse, r2])  

result_dataset_itog = pd.DataFrame(result_dataset_itog, columns = ['Dataset', 'Target', 'Model', 'MAE', 'MSE', 'R2']) 

In [ ]:
result_dataset_itog.sort_values(by = ['Target','R2'])

#### Выводы: 
    
Для дальнейшей работы решено взять датасет с удалением выбросов, используя межквартильный размах, и преобразованием StandardScaler 
     

# Обучение моделей

In [ ]:
# Скопируем датасет и сразу удалим выбросы
df_work = df_united.copy()

df_work['Угол нашивки, град'] = df_work['Угол нашивки, град'].astype('category') 

df_work = drop_blowout(df_work, 'IQR') 
print()
df_work = drop_blowout(df_work, 'IQR')
print()
df_work = drop_blowout(df_work, 'IQR')

In [ ]:
# sns.displot(df_work['Модуль упругости, ГПа'], kind="kde")

In [ ]:
# Переведем 'Прочность при растяжении, МПа' в гигапаскаль и поменяем название  
# df_work['Прочность при растяжении, МПа'] = df_work['Прочность при растяжении, МПа'] / 1000
# df_work.rename(columns = {'Прочность при растяжении, МПа':'Прочность при растяжении, ГПа'}, inplace = True)

In [ ]:
# Разобьем датасет на входящие и целевые переменные 
X = df_work.drop(['Модуль упругости при растяжении, ГПа', 
                  'Прочность при растяжении, МПа',
                  'Соотношение матрица-наполнитель'], axis = 1)
X1 = X.copy()

y1 = df_work['Модуль упругости при растяжении, ГПа']

X2 = X.copy()

y2 = df_work['Прочность при растяжении, МПа']

In [ ]:
# Разобьем датасеты на обучающую и тестовую выборки
X1_train, X1_test, y1_train, y1_test = train_test_split (X1, y1, test_size = 0.3, random_state = 13)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

X2_train, X2_test, y2_train, y2_test = train_test_split (X2, y2, test_size = 0.3, random_state = 13)
X2_train.shape, X2_test.shape, y2_train.shape, y2_test.shape

In [ ]:
# Функция получения числовых, категориальных колонок... 
def get_num_cat_pw_col(dataset):
    
    num_col = [i for i in dataset.columns if dataset[i].dtype in ['int64','float64'] 
               and stats.skew(dataset[i], axis=0, bias=True) < 0.3 
               and stats.skew(dataset[i], axis=0, bias=True) > -0.3]  
    cat_col = [i for i in dataset.columns if dataset[i].dtype in ['object','category']]
    
    pw_col = [i for i in dataset.columns if dataset[i].dtype in ['int64','float64'] 
              and (stats.skew(dataset[i], axis=0, bias=True) > 0.3 
              or stats.skew(dataset[i], axis=0, bias=True) < -0.3)] 
    
    return num_col, cat_col, pw_col

In [ ]:
num_col, cat_col, pw_col = get_num_cat_pw_col(X)
num_col, cat_col, pw_col

In [ ]:
# Сделаем препроцессор данных
preprocessor = ColumnTransformer(transformers=[
                                                ('num', StandardScaler(), num_col),
                                                ('pw', PowerTransformer(method = 'yeo-johnson'), pw_col),
                                                ('cat', OneHotEncoder(handle_unknown='ignore'), cat_col)                
                                              ])

### Модуль упругости при растяжении, ГПа

In [ ]:
# Сделаем подбор гиперпараметров по сетке, т.к. м уже получили оценки моделей при использовании параметров по умолчанию 

X_search = preprocessor.fit_transform(X1_train) # нормализованный датасет для подбора гиперпараметров

LinReg_parameters = {
                     'fit_intercept' : [True, False],
                     'n_jobs' : np.arange(1, 10),
                     'positive' : [True, False]
                    }
reg = GridSearchCV(LinearRegression(), LinReg_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y1_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

DecTreeReg_parameters = {
                         'criterion' : ['squared_error', 'absolute_error'],
                         'max_depth' : np.arange(1, 20),
                         'random_state' : [13]
                        }
reg = GridSearchCV(DecisionTreeRegressor(), DecTreeReg_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y1_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

LinSVR_parameters = {
                     'epsilon' : np.arange(0, 10, 0.1),
                     'loss' : ['epsilon_insensitive', 'squared_epsilon_insensitive'],
                     'fit_intercept' : [True, False],
                     'random_state' : [13]
                    }
reg = GridSearchCV(LinearSVR(), LinSVR_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y1_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

KNNReg_parameters = {
                     'n_neighbors' : np.arange(1,300),
                     'weights' : ['uniform', 'distance'],
                     'p' : [1,2]
                    }
reg = GridSearchCV(KNeighborsRegressor(), KNNReg_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y1_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

RandForReg_parameters = {
                         'criterion' : ['squared_error', 'absolute_error'],
                         'max_depth' : np.arange(1, 20),
                         'random_state' : [13]
                        }
reg = GridSearchCV(RandomForestRegressor(), RandForReg_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y1_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

In [ ]:
# Сделаем словарь для удобства вызова моделей с наилучшими гиперпараметрами 
dict_model_t1 = {
                 'LinReg' : LinearRegression(fit_intercept = True, 
                                             n_jobs = 1, 
                                             positive = False), 
                 'DecTreeReg' : DecisionTreeRegressor(criterion = 'absolute_error', 
#                                                      max_depth = 1, 
                                                      random_state = 13),
                 'LinSVR': LinearSVR(epsilon = 0.6, 
                                     fit_intercept = True, 
                                     loss = 'squared_epsilon_insensitive', 
                                     random_state = 13),
                 'KNNReg': KNeighborsRegressor(n_neighbors = 271, 
                                               weights = 'uniform',
                                               p = 2),              
                 'RandForReg' : RandomForestRegressor(criterion = 'squared_error', 
#                                                      max_depth = 1, 
                                                      random_state = 13)
                }

In [ ]:
# Обучим модели и сохраним результаты
result_model1 = []
for model_name, model in dict_model_t1.items():           
        
    # Соединим препроцессор и модель в Pipeline
    regressor = Pipeline(steps=[('preprocessor', preprocessor),
                                ('model', model)],
                         verbose=True)
    
    # Обучение
    regressor.fit(X1_train, y1_train)
    
    # Получаем предсказания 
    y_pred = regressor.predict(X1_test)
    
    # Cнимаем метрики 
    mae = mean_absolute_error(y1_test, y_pred)
    mse = mean_squared_error(y1_test, y_pred)
    r2 = r2_score(y1_test, y_pred)
    result_model1.append([model_name, mae, mse, r2])     
    
    # Визуализация работы модели
    fig = plt.figure(figsize = (10,3))
    ax = sns.lineplot(data = np.array(y1_test), label = 'true values', c = 'orange')
    ax = sns.lineplot(data = y_pred, label = 'predicted values', c = 'darkgreen')
    ax = plt.title(model_name)
    ax = plt.legend(loc = 'best')
    plt.show()
    
    # Cохранение моделей 
    pickle.dump(regressor, open(f'{current_path}\\content\\{model_name}_t1.pkl','wb'))

result_model1 = pd.DataFrame(result_model1, columns = ['Model', 'MAE', 'MSE', 'R2']) 

In [ ]:
result_model1

### Прочность при растяжении, МПа

In [ ]:
# Сделаем подбор гиперпараметров по сетке, т.к. м уже получили оценки моделей при использовании параметров по умолчанию 

X_search = preprocessor.fit_transform(X2_train) # нормализованный датасет для подбора гиперпараметров

scoring = make_scorer(r2_score)

LinReg_parameters = {
                     'fit_intercept' : [True, False],
                     'n_jobs' : np.arange(1, 10),
                     'positive' : [True, False]
                    }
reg = GridSearchCV(LinearRegression(), LinReg_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y2_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

DecTreeReg_parameters = {
                         'criterion' : ['squared_error', 'absolute_error'],
                         'max_depth' : np.arange(1, 20),
                         'random_state' : [13]
                        }
reg = GridSearchCV(DecisionTreeRegressor(), DecTreeReg_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y2_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

LinSVR_parameters = {
                     'epsilon' : np.arange(0, 10, 0.1),
                     'loss' : ['epsilon_insensitive', 'squared_epsilon_insensitive'],
                     'fit_intercept' : [True, False],
                     'random_state' : [13]
                    }
reg = GridSearchCV(LinearSVR(), LinSVR_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y2_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

KNNReg_parameters = {
                     'n_neighbors' : np.arange(1,300),
                     'weights' : ['uniform', 'distance'],
                     'p' : [1,2]
                    }
reg = GridSearchCV(KNeighborsRegressor(), KNNReg_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y2_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

RandForReg_parameters = {
                         'criterion' : ['squared_error', 'absolute_error'],
                         'max_depth' : np.arange(1, 20),
                         'random_state' : [13]
                        }
reg = GridSearchCV(RandomForestRegressor(), RandForReg_parameters, cv = 5, scoring = scoring)
search = reg.fit(X_search, y2_train)
print(f'Параметры для {search.estimator} - {search.best_params_}')

In [ ]:
# Сделаем словарь для удобства вызова моделей с наилучшими гиперпараметрами 
dict_model_t2 = {
                 'LinReg' : LinearRegression(fit_intercept = True, 
                                             n_jobs = 1, 
                                             positive = True), 
                 'DecTreeReg' : DecisionTreeRegressor(criterion = 'squared_error', 
#                                                      max_depth = 1, 
                                                      random_state = 13),
                 'LinSVR': LinearSVR(epsilon = 1.8, 
                                     fit_intercept = True, 
                                     loss = 'squared_epsilon_insensitive', 
                                     random_state = 13),
                 'KNNReg': KNeighborsRegressor(n_neighbors = 258, 
                                               weights = 'uniform',
                                               p = 2),              
                 'RandForReg' : RandomForestRegressor(criterion = 'squared_error', 
#                                                      max_depth = 1, 
                                                      random_state = 13)
                }

In [ ]:
# Обучим модели и сохраним результаты
result_model2 = []
for model_name, model in dict_model_t2.items():           
        
    # Соединим препроцессор и модель в Pipeline
    regressor = Pipeline(steps=[('preprocessor', preprocessor),
                                ('model', model)],
                         verbose=True)
    
    # Обучение
    regressor.fit(X2_train, y2_train)
    
    # Получаем предсказания 
    y_pred = regressor.predict(X2_test)
    
    # Cнимаем метрики 
    mae = mean_absolute_error(y2_test, y_pred)
    mse = mean_squared_error(y2_test, y_pred)
    r2 = r2_score(y2_test, y_pred)
    result_model2.append([model_name, mae, mse, r2])     

    # Визуализация работы модели
    fig = plt.figure(figsize = (10,3))
    ax = sns.lineplot(data = np.array(y2_test), label = 'true values', c = 'orange')
    ax = sns.lineplot(data = y_pred, label = 'predicted values', c = 'darkgreen')
    ax = plt.title(model_name)
    ax = plt.legend(loc = 'best')
    plt.show()
    
    # Cохранение моделей 
    pickle.dump(regressor, open(f'{current_path}\\content\\{model_name}_t2.pkl','wb'))

result_model2 = pd.DataFrame(result_model2, columns = ['Model', 'MAE', 'MSE', 'R2']) 

In [ ]:
result_model2

In [ ]:
# Проверка модели
regressor = pickle.load(open(f'{current_path}\\content\\DecTreeReg_t2.pkl','rb'))
y_pred = regressor.predict(X2_test)
print ('MAE =',mean_absolute_error(y2_test, y_pred))
print ('MSE =',mean_squared_error(y2_test, y_pred))
print ('R2 =',r2_score(y2_test, y_pred))

# Обучение  нейронной сети

###  Соотношение матрица-наполнитель

In [ ]:
# Разобьем датасет на обучающую и тестовую выборки
X3 = df_work.drop('Соотношение матрица-наполнитель', axis = 1) 
y3 = df_work['Соотношение матрица-наполнитель']

X3_train, X3_test, y3_train, y3_test = train_test_split (X3, y3, test_size = 0.3, random_state = 13)
X3_train.shape, X3_test.shape, y3_train.shape, y3_test.shape

In [ ]:
num_col, cat_col, pw_col = get_num_cat_pw_col(X3_train)
num_col, cat_col, pw_col

In [ ]:
# Сделаем препроцессоры данных
preprocessorNN = ColumnTransformer(transformers=[
                                                ('num', StandardScaler(), num_col),
                                                ('pw', PowerTransformer(method = 'yeo-johnson'), pw_col),
                                                ('cat', OneHotEncoder(handle_unknown='ignore'), cat_col)                
                                              ])
num_col
preprocessorNN2 = ColumnTransformer(transformers=[
                                                ('num', MinMaxScaler(), num_col+pw_col),
                                                ('cat', OneHotEncoder(handle_unknown='ignore'), cat_col)                
                                              ])

In [ ]:
preprocessorNN = preprocessorNN.fit(X3_train)
preprocessorNN2 = preprocessorNN2.fit(X3_train)

In [ ]:
# Cохранение препроцессоров 
pickle.dump(preprocessorNN, open(f'{current_path}\\content\\modelNS\\preprocessorNN2.pkl','wb'))
pickle.dump(preprocessorNN2, open(f'{current_path}\\content\\modelNS\\preprocessorNN2.pkl','wb'))

In [ ]:
X3_train = preprocessorNN.transform(X3_train)
X3_test = preprocessorNN.transform(X3_test)
# X3_train = preprocessorNN2.transform(X3_train)
# X3_test = preprocessorNN2.transform(X3_test)

In [ ]:
# X3_train['Угол нашивки, град'] = utils.to_categorical(X3_train['Угол нашивки, град'])
# X3_test['Угол нашивки, град'] = utils.to_categorical(X3_test['Угол нашивки, град'])

In [ ]:
ordno = 0

In [ ]:
# Начнем подбирать параметры НС
model = None
ordno += 1

# Оптимизаторы
adam = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
sgd = SGD(learning_rate=0.01, momentum=0.0, nesterov=False)
rms = RMSprop() 

# Модель НС
model = Sequential()
# model.add(BatchNormalization(input_dim = 12))
model.add(Dense(8, input_dim = 13, kernel_initializer='normal', activation = 'relu')) 
# model.add(Dense(8, kernel_initializer='normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))

model.compile(loss = 'mean_squared_error', 
              optimizer = adam, 
              metrics = ['mean_squared_error']) 

# Сохранение лучшей модели
modelcheckpoint_list = ModelCheckpoint(f'{current_path}\\content\\modelNS\\Best_model{ordno}.hdf5', 
                                       monitor='val_loss', 
                                       verbose = 1, 
                                       save_best_only = True, 
                                       mode ='min')
# Ранная остановка
earlystop_list = EarlyStopping(monitor = 'val_loss', 
                               mode = 'min', 
                               verbose = 1, 
                               patience = 20)

history = model.fit(X3_train, y3_train, 
                    batch_size = 32, 
                    epochs = 1000,
                    shuffle = True,
                    validation_split = 0.1,
                    callbacks = [earlystop_list, modelcheckpoint_list],
                    verbose = 1) 

In [ ]:
# fig = plt.figure(figsize = (5,3))
# ax =plt.plot(history.history['loss'], 
#          label='loss на обучающем наборе')
# ax = plt.plot(history.history['val_loss'], 
#          label='loss на проверочном наборе')
# ax = plt.xlabel('Эпоха обучения')
# ax = plt.ylabel('loss')
# ax = plt.legend()
# plt.show()

In [ ]:
# Получим метрики сохраненных моделей - StandardScaler
result_model3 = []
for i in range(1,35):
    saved_model = load_model(f'{current_path}\\content\\modelNS\\Best_model{i}.hdf5')
    # Получаем предсказания 
    y_pred = saved_model.predict(X3_test)

    # Cнимаем метрики 
    mae = mean_absolute_error(y3_test, y_pred)
    mse = mean_squared_error(y3_test, y_pred)
    r2 = r2_score(y3_test, y_pred)      
    result_model3.append([f'Best_model{i}', mae, mse, r2])  
    
result_model3 = pd.DataFrame(result_model3, columns = ['Model', 'MAE', 'MSE', 'R2'])       

In [ ]:
result_model3.sort_values(by = ['R2']) # StandardScaler

In [ ]:
# Получим метрики сохраненных моделей - MinMaxScaler
result_model4 = []
for i in range(1,35):
    saved_model = load_model(f'{current_path}\\content\\modelNS\\Best_model{i}.hdf5')
    # Получаем предсказания 
    y_pred = saved_model.predict(X3_test)

    # Cнимаем метрики 
    mae = mean_absolute_error(y3_test, y_pred)
    mse = mean_squared_error(y3_test, y_pred)
    r2 = r2_score(y3_test, y_pred)      
    result_model4.append([f'Best_model{i}', mae, mse, r2])  
    
result_model4 = pd.DataFrame(result_model4, columns = ['Model', 'MAE', 'MSE', 'R2'])             

In [ ]:
result_model4.sort_values(by = ['R2']) # MimMaxScaler

In [ ]:
best_model = load_model(f'{current_path}\\content\\modelNS\\Best_model8.hdf5')

best_model.summary()

# Получаем предсказания 
y_pred = best_model.predict(X3_test)

# Cнимаем метрики 
mae = mean_absolute_error(y3_test, y_pred)
mse = mean_squared_error(y3_test, y_pred)
r2 = r2_score(y3_test, y_pred)
mae, mse, r2  

# Сохранение весов и модели в другом формате
best_model.save_weights(f'{current_path}\\content\\modelNS\\Weights_final_model.h5')
best_model.save(f'{current_path}\\content\\modelNS\\Final_model.h5')

In [ ]:
# Визуализация работы НС
fig = plt.figure(figsize = (10,3))
ax = sns.lineplot(data = np.array(y3_test), label = 'true values', c = 'orange')
ax = sns.lineplot(data = y_pred[:,0], label = 'predicted values', c = 'darkgreen')
ax = plt.legend(loc = 'best')
plt.show()

### Подбор параметров НС перебором

In [ ]:
layer_list = [1,2,3]
neuron_list = [8,16,32,64]
activation_list = ['tanh','linear','relu']
optimizer_list = ['sgd','adam']

for layer in layer_list:
    
    for neuron in neuron_list: 
        
        for funcactiv in activation_list:
            
            for optim in optimizer_list:
                
                # Модель НС
                model = Sequential()
                model.add(Dense(neuron, input_dim = 13, kernel_initializer='normal', activation = funcactiv)) 
                
                if layer > 1:                 
                    model.add(Dense(neuron, kernel_initializer='normal', activation = funcactiv))
                
                if layer > 2: 
                    model.add(Dense(neuron, kernel_initializer='normal', activation = funcactiv))
                
                model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))

                model.compile(loss = 'mean_squared_error', 
                              optimizer = optim, 
                              metrics = ['mean_squared_error']) 

                # Сохранение лучшей модели
                modelcheckpoint_list = ModelCheckpoint(f'{current_path}\\content\\modelNS\\СycleEnum3\\{layer}_{neuron}_{funcactiv}_{optim}.hdf5', 
                                                       monitor='val_loss', 
                                                       verbose = 0, 
                                                       save_best_only = True, 
                                                       mode ='min')
                # Ранная остановка
                earlystop_list = EarlyStopping(monitor = 'val_loss', 
                                               mode = 'min', 
                                               verbose = 0, 
                                               patience = 20)

                history = model.fit(X3_train, y3_train, 
                                    batch_size = 32, 
                                    epochs = 1000,
                                    shuffle = True,
                                    validation_split = 0.1,
                                    callbacks = [earlystop_list, modelcheckpoint_list],
                                    verbose = 0)     

In [ ]:
# Получим метрики сохраненных моделей - StandardScaler
result_model5 = []
for layer in layer_list:
    
    for neuron in neuron_list: 
        
        for funcactiv in activation_list:
            
            for optim in optimizer_list:
    
                saved_model = load_model(f'{current_path}\\content\\modelNS\\СycleEnum\\{layer}_{neuron}_{funcactiv}_{optim}.hdf5')

                # Получаем предсказания 
                y_pred = saved_model.predict(X3_test)

                # Cнимаем метрики 
                mae = mean_absolute_error(y3_test, y_pred)
                mse = mean_squared_error(y3_test, y_pred)
                r2 = r2_score(y3_test, y_pred)      
                result_model5.append([f'{layer}_{neuron}_{funcactiv}_{optim}', mae, mse, r2])  

In [ ]:
result_model5 = pd.DataFrame(result_model5, columns = ['Model', 'MAE', 'MSE', 'R2'])     
result_model5.sort_values(by = ['R2']).tail(20)

In [ ]:
# Получим метрики сохраненных моделей - MinMaxScaler
result_model6 = []
for layer in layer_list:
    
    for neuron in neuron_list: 
        
        for funcactiv in activation_list:
            
            for optim in optimizer_list:
    
                saved_model = load_model(f'{current_path}\\content\\modelNS\\СycleEnum2\\{layer}_{neuron}_{funcactiv}_{optim}.hdf5')

                # Получаем предсказания 
                y_pred = saved_model.predict(X3_test)

                # Cнимаем метрики 
                mae = mean_absolute_error(y3_test, y_pred)
                mse = mean_squared_error(y3_test, y_pred)
                r2 = r2_score(y3_test, y_pred)      
                result_model6.append([f'{layer}_{neuron}_{funcactiv}_{optim}', mae, mse, r2])  

In [ ]:
result_model6 = pd.DataFrame(result_model6, columns = ['Model', 'MAE', 'MSE', 'R2'])     
result_model6.sort_values(by = ['R2']).tail(20)